# **【注意】**使用模型的时候记得将运行时类型改为GUP

In [20]:
!git clone https://github.com/prs-eth/U-TILISE.git

Cloning into 'U-TILISE'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 64 (delta 8), reused 63 (delta 7), pack-reused 0
Receiving objects: 100% (64/64), 9.61 MiB | 26.82 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
!nvcc --version

In [ ]:
# !pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install torchdata
!pip install torchtext

In [23]:
!pip install -r requ.txt

# !pip install h5py
# !pip install ipywidgets
# !pip install ipykernel
# !pip install kornia
# !pip install matplotlib
# !pip install numba
# !pip install omegaconf
# !pip install tqdm
# !pip install wandb
# !pip install setuptools
# !pip install prodict
# !pip install torchgeometry
# !pip install tensorboard
# !pip install nestargs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.3 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=912e5954188ce4617f91a4cd227ff8

In [21]:
%cd /content/U-TILISE

/content/U-TILISE


In [5]:
!pip install torchinfo

In [ ]:
# !pip install -U numba

In [ ]:
import os
import sys
import torch
from matplotlib import pyplot as plt
from typing import Any, Dict

# /content/U-TILISE
from lib import config_utils
from lib import data_utils
from lib import visutils
from lib.eval_tools import (
    Imputation,
    visualize_att_for_one_head_across_time,
    visualize_att_for_target_t_across_heads
)

In [ ]:
!bash ./scripts/download_checkpoints.sh

In [8]:
# GPU高RAM模式下下载快一点,20分钟左右,CPU下载要半小时。
# 下载完还要解压，解压也需要一段时间。
!bash ./scripts/download_data_earthnet2021.sh

--2023-11-02 09:07:22--  https://share.phys.ethz.ch/~pf/stuckercdata/u-tilise/data/earthnet2021_iid.tar.gz
Resolving share.phys.ethz.ch (share.phys.ethz.ch)... 129.132.80.27, 2001:67c:10ec:3dc4::39
Connecting to share.phys.ethz.ch (share.phys.ethz.ch)|129.132.80.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19426030503 (18G) [application/x-gzip]
Saving to: ‘earthnet2021_iid.tar.gz’

earthnet2021_iid.ta 100%[===================>]  18.09G  21.3MB/s    in 15m 20s 

2023-11-02 09:22:42 (20.1 MB/s) - ‘earthnet2021_iid.tar.gz’ saved [19426030503/19426030503]


In [9]:
def get_dataloader_testdata(
    config_file_train: str,
    config_file_test: str,
    run_mode: str = 'test'
) -> torch.utils.data.dataloader.DataLoader:
    if not os.path.isfile(config_file_train):
        raise FileNotFoundError(f'Cannot find the configuration file used during training: {config_file_train}\n')

    if not os.path.isfile(config_file_test):
        raise FileNotFoundError(f'Cannot find the test configuration file: {config_file_test}\n')

    # Read the configuration file used during training
    config = config_utils.read_config(config_file_train)

    # Merge generic data settings (used during training) with test-specific data settings
    config_testdata = config_utils.read_config(config_file_test)
    config.data.update(config_testdata.data)
    if 'mask' in config_testdata:
        config.mask.update(config_testdata.mask)
    config.misc.run_mode = run_mode

    # Get the data loader
    dset = data_utils.get_dataset(config, phase=run_mode)
    dataloader = torch.utils.data.DataLoader(
        dataset=dset, batch_size=1, shuffle=False, num_workers=8, drop_last=False
    )

    return dataloader


def get_sample(
    dataloader: torch.utils.data.dataloader.DataLoader,
    sample_index: int
) -> Dict[str, Any]:

    batch = dataloader.dataset.__getitem__(sample_index)

    # Introduce the batch dimension (required for the forward pass)
    for k, v in batch.items():
        if isinstance(v, torch.Tensor):
            batch[k] = v.unsqueeze(0)
        elif isinstance(v, int):
            batch[k] = [v]

    return batch

## Imputation of satellite image time series with real data gaps

<hr style="height: 2px;" />

**Settings**

In [10]:
# Default data and model settings (i.e., settings used during training)
config_file_train = 'configs/demo.yaml'

# Test-specific data settings
config_file_test = 'configs/config_earthnet2021_test.yaml'

# Model weights
checkpoint = 'checkpoints/utilise_earthnet2021.pth'

**Settings**

In [11]:
dataloader = get_dataloader_testdata(config_file_train, config_file_test)

Instantiate U-TILISE

In [12]:
utilise = Imputation(config_file_train, method='utilise', checkpoint=checkpoint)

Checkpoint 'checkpoints/utilise_earthnet2021.pth' loaded.
Chosen epoch: 982


# 获取GEE数据


In [1]:
import ee

# 进行身份验证
ee.Authenticate()
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=rFQHR7_ejSLMmuxImBVRlKxSKCEDWcdqxEfeFXDHUJU&tc=_s3zGd9NSwdy-CSKjnVxTjqrq3sVhe6tjxd9gqOG6HY&cc=yrpEoAfLr_C09X0Lgma8_jREV_5-abfYNKEYV-hfM-s

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmKmoiGuyM9IP8u7Kj4-MFpjE2JociI7_r5iFHAtdqdWk5pfCcFJw8

Successfully saved authorization token.


In [2]:
import numpy as np
# import tensorflow as tf
import torch

*1.* 根据位置信息，获取GEE图像集合

In [3]:
# 加载 FAO/GAUL/2015/level2 数据集
gaul = ee.FeatureCollection('FAO/GAUL/2015/level2');

# 选择深圳的区域，根据 ADM2_NAME 属性过滤
shenzhen = gaul.filter(ee.Filter.eq('ADM2_NAME', 'Shenzhen'));

# 获取深圳区域的几何信息
shenzhenGeometry = shenzhen.geometry();

In [4]:
# 位置
polygon = [ [113.93631717340367,22.60000605357884],
        [114.04137393609898,22.60000605357884],
        [114.04137393609898,22.695060891235958],
        [113.93631717340367,22.695060891235958],
        [113.93631717340367,22.60000605357884]
      ]
region = ee.Geometry.Polygon(polygon)

In [5]:
# 图像集合
collection = ee.ImageCollection('COPERNICUS/S2')\
        .filterDate('2020-09-01', '2020-10-01')\
        .filterBounds(region)
print(collection.getInfo())

{'type': 'ImageCollection', 'bands': [], 'id': 'COPERNICUS/S2', 'version': 1698930770289040.0, 'properties': {'date_range': [1435017600000, 1647993600000], 'period': 0, 'system:visualization_0_min': '0.0', 'type_name': 'ImageCollection', 'keywords': ['copernicus', 'esa', 'eu', 'msi', 'radiance', 'sentinel'], 'system:visualization_0_bands': 'B4,B3,B2', 'thumb': 'https://mw1.google.com/ges/dd/images/sentinel2_thumb.png', 'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 data contain 13 UINT16 spectral bands representing\nTOA reflectance scaled by 10000. See the <a href="https://sentinel.esa.int/documents/247904/685211/Sentinel-2_User_Handbook">Sentinel-2 User Handbook</a>\nfor details. In addition, three QA bands are present where one\n(QA60) is a bitm

*2.* 从图像集合中筛选像素覆盖率为100%的图像



In [6]:
# 求图像集合的中图像的geometry，与自己定义的region区域进行计算，求region是否包含在图像的geometry中。
# 如果包含则表示该图像能够百分百覆盖region区域，否则不能包含region区域，该图像将被筛除。
collectionList = collection.toList(collection.size().getInfo())
# print(collectionList.size().getInfo()) # 12
# print(type(collectionList.getInfo()[0])) # <class 'dict'>
# print(type(collectionList.get(1))) # <class 'ee.computedobject.ComputedObject'>
# print(collectionList.get(1).getInfo()) # {'type': 'Image', 'bands': [...], ......}

# 遍历imageList，计算图像是否百分百覆盖region
def isFullCovered(image):
  image_geometry = ee.Image(image).geometry()
  isContainedIn = region.containedIn(image_geometry) # 小范围是否包含在大范围中
  return isContainedIn

isFullCoveredList = collectionList.map(isFullCovered)
# print(isFullCoveredList.getInfo()) # [False, True, False, True, False, True, False, True, False, True, False, True]
# print(type(isFullCoveredList)) # <class 'ee.ee_list.List'>

isFullCoveredList = isFullCoveredList.getInfo()
print(isFullCoveredList) # [False, True, False, True, False, True, False, True, False, True, False, True]
print(type(isFullCoveredList)) # <class 'list'>

[False, True, False, True, False, True, False, True, False, True, False, True]
<class 'list'>


In [7]:
# 获取图像集合中每个图像的'system:time_start'属性
def getTimeStart(image):
  # image_time_start = ee.Image(image).get('system:time_start')
  # isContainedIn = region.containedIn(image_geometry) # 小范围是否包含在大范围中
  return ee.Date(ee.Image(image).get('system:time_start')).format('YYYY-MM-dd')
timeStartList = collectionList.map(getTimeStart)
# print(timeStartList.getInfo()) # ['2020-09-01', '2020-09-01', '2020-09-06', '2020-09-06', '2020-09-11', '2020-09-11', '2020-09-16', '2020-09-16', '2020-09-21', '2020-09-21', '2020-09-26', '2020-09-26']
# print(type(timeStartList)) # <class 'ee.ee_list.List'>

timeStartList = timeStartList.getInfo()
# print(type(timeStartList)) # <class 'list'>

In [8]:
# 获取图像集合中每个图像的 id
def getImageId(image):
  # image_time_start = ee.Image(image).get('system:time_start')
  # isContainedIn = region.containedIn(image_geometry) # 小范围是否包含在大范围中
  return ee.Image(image).id()
imageIdList = collectionList.map(getImageId)
# print(imageIdList.getInfo()) # ['20200901T025549_20200901T030025_T49QGF', '20200901T025549_20200901T030025_T49QHF', '20200906T025551_20200906T031003_T49QGF', '20200906T025551_20200906T031003_T49QHF', '20200911T025549_20200911T030838_T49QGF', '20200911T025549_20200911T030838_T49QHF', '20200916T025551_20200916T030959_T49QGF', '20200916T025551_20200916T030959_T49QHF', '20200921T025549_20200921T030950_T49QGF', '20200921T025549_20200921T030950_T49QHF', '20200926T025551_20200926T030132_T49QGF', '20200926T025551_20200926T030132_T49QHF']
# print(type(imageIdList)) # <class 'ee.ee_list.List'>

imageIdList = imageIdList.getInfo()
# print(type(imageIdList)) # <class 'list'>

In [9]:
# 根据 isFullCoveredList、timeStartList、imageIdList筛选出覆盖率为100%且时间不重叠的图像，并保存在字典中，字典的键表示时间，值表示图像id
validImageDict = {}
validImageList = []
for i in range(len(imageIdList)):
  if isFullCoveredList[i]: # 覆盖率为100%
    time_start = timeStartList[i]
    if time_start not in validImageDict: # 未存在该时间的图像
      validImageDict[time_start] = imageIdList[i]
      validImageList.append(ee.Image(collectionList.get(i)))

# print(validImageDict) # {'2020-09-01': '20200901T025549_20200901T030025_T49QHF', '2020-09-06': '20200906T025551_20200906T031003_T49QHF', '2020-09-11': '20200911T025549_20200911T030838_T49QHF', '2020-09-16': '20200916T025551_20200916T030959_T49QHF', '2020-09-21': '20200921T025549_20200921T030950_T49QHF', '2020-09-26': '20200926T025551_20200926T030132_T49QHF'}
# print(len(validImageDict)) # 6
# print(validImageList) # [<ee.image.Image object at 0x7a46c1db84f0>, <ee.image.Image object at 0x7a46c1dbb220>, <ee.image.Image object at 0x7a46c1dba200>, <ee.image.Image object at 0x7a46c1db9d80>, <ee.image.Image object at 0x7a46c1dbb7f0>, <ee.image.Image object at 0x7a46c1dbb670>]
# print(len(validImageList)) # 6


*3.* 根据 QA60 波段计算云掩膜，并将云掩膜作为一个波段添加到原始图像的最后

> **注意：这里的云掩膜与模型中需要的云掩膜相反。**
*   这里的得到的云掩膜，值为0处，是黑色，表示无效的有云区域，值为1处，是白色，表示有效的无云区域。
*   模型中需要的云掩膜，值为0处，是黑色，表示无云覆盖的无云区域，值为1处，是白色，表示有云覆盖的有云区域。


In [10]:
# validImageList = ee.List(validImageList)
# print(validImageList.getInfo()) # [{'type': 'Image', 'bands': [......], ......}]
# print(type(validImageList)) # <class 'ee.ee_list.List'>
# print(validImageList.get(1).getInfo()) # {'type': 'Image', 'bands': [......], ......}

# QA10和QA20波段是空的，使用QA60波段计算的云掩膜，包含不透明的云和卷云的标记，Bit10标记不透明的云（0为无云，1为有云），Bit11标记卷云（0为无云，1为有云）。
# 使用QA60波段计算的云掩膜，由于只包含不透明的云和卷云的标记，效果比较差，存在识别不完全的情况，对薄一点的云/雾去识别不出来。
def getCloudMaskByQA60(image):
  qa = ee.Image(image).select(['QA60'],['cloud_mask'])

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  # .eq(0)为True表示有效位置（无云区域），为False表示无效位置（有云区域）
  # 使用这个方法得到的云掩膜对薄云识别不佳，效果不好
  mask = ee.Image(qa.bitwiseAnd(cloudBitMask).eq(0))\
        .And(ee.Image(qa.bitwiseAnd(cirrusBitMask).eq(0)))
  # return image.updateMask(mask)\
  #     .select("B.*")\
  #     .copyProperties(image, ["system:time_start"])

  # 注意：
  # 返回的结果黑色是有云的区域，是0，表示无效区域，白色是1，表示有效区域。
  # 返回的结果和模型中需要的云掩膜相反，模型中需要的云掩膜是黑色，值为0，表示未被云覆盖区域，白色，值为1，表示云覆盖区域。
  return mask

def addCloudMask(i):
  image = ee.List(validImageList).get(i)
  cloud_mask = ee.Image(cloudMaskList.get(i)).int16()
  image = ee.Image(image).addBands(cloud_mask,['cloud_mask'])
  return image
# 将云掩膜作为一个波段，添加云掩膜波段到原始图像的最后
cloudMaskList = ee.List(validImageList).map(getCloudMaskByQA60)
# print(type(cloudMaskList)) # <class 'ee.ee_list.List'>
# print(cloudMaskList.size().getInfo()) # 6
# print(cloudMaskList.getInfo()) # [{'type': 'Image', 'bands': [......], ......}]

imageAddCloudMaskList = ee.List.sequence(0,len(validImageList)-1).map(addCloudMask)
# print(imageAddCloudMaskList.size().getInfo()) # 6
# print(len(imageAddCloudMaskList.get(0).getInfo()['bands'])) # 17
# print(imageAddCloudMaskList.get(0).getInfo()['bands'][-1]) # {'id': 'cloud_mask', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32649', 'crs_transform': [60, 0, 799980, 0, -60, 2600040]}


In [ ]:
# # 将云掩膜叠加到原始图像中的最后一层
# # for循环很慢
# isContainedInList = []
# coveredList = []
# for i in range(imageList.size().getInfo()):
#   image = imageList.get(i)
#   image_geometry = ee.Image(image).geometry()
#   isContainedIn = region.containedIn(image_geometry).getInfo() # 小范围是否包含在大范围中
#   isContainedInList.append(isContainedIn)
#   if isContainedIn:
#     coveredList.append(ee.Image(image))
# # print(isContainedInList) # [False, True, False, True, False, True, False, True, False, True, False, True]
# # print(len(coveredList)) # 6

显示一个图像的掩膜结果

In [11]:
# !pip install geemap
img = ee.Image('COPERNICUS/S2/20200901T025549_20200901T030025_T49QHF')
image_mask = getCloudMaskByQA60(img)

import geemap
m = geemap.Map()
m.addLayer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'img')
m.addLayer(ee.Image(image_mask), {'min': 0, 'max': 1}, 'image_mask') # 黑色区域是云
m.addLayer(img.updateMask(image_mask), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'image_masked') # 有云的地方是空白
display(m)

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

*4.* 将包含云掩膜的有效时间序列图像上传到 Cloud Storage

现在有的数据：

1. region（ee.Geometry 类型）：图像位置

2. collectionList（ee.List 类型）：根据日期、位置筛选的原始图像列表

3. isFullCoveredList（list 类型）：原始图像的像素覆盖率是否百分百覆盖的布尔列表

4. timeStartList（list 类型）：与原始图像列表的对应的开始时间列表

5. imageIdList（list 类型）：与原始图像列表的对应的图像 id 列表

6. validImageDict（dict 类型）：有效时间序列图像的“日期 -- 图像 id”字典

7. validImageList（list 类型）：有效时间序列原始图像列表

8. cloudMaskList（ee.List 类型）：有效时间序列原始图像对应的云掩膜列表

9. imageAddCloudMaskList（ee.List 类型）：包含云掩膜的有效时间序列图像列表



In [12]:
BUCKET = 'u-tilise'
FOLDER = 'shenzhen_test'
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
kernel_buffer = [128, 128]

buffer_region = region.buffer(distance=10*KERNEL_SIZE).getInfo()['coordinates'] # 10 是导出时的分辨率 scale
regionInfo = region.getInfo()['coordinates']

In [ ]:
image_size = collection.first().clip(region).getInfo()['bands'][1]['dimensions']
print(image_size) # [808, 1076]

[808, 1076]


上传数据

In [13]:
# 上传数据到 Cloud Storage，不采用 kernel_buffer，直接把原图的所有波段导出
def exportImageToCloudStorage(out_image_base, image):
  task = ee.batch.Export.image.toCloudStorage(
    image = image.int16(),
    # image = s2image.select(BANDS),
    description = out_image_base,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/shenzhen_tif_202009/' + out_image_base,
    region = regionInfo,
    # scale = 30,
    scale = 10,
    # fileFormat = 'TFRecord', # 默认为tif格式
    maxPixels = 1e10,
    # formatOptions = {
    #   'patchDimensions': KERNEL_SHAPE,
    #   # 'kernelSize': kernel_buffer,
    #   'compressed': True,
    #   'maxFileSize': 104857600
    # }
  )

  task.start()

# 由于上传时需要传递参数，参数要通过.getInfo()方法来获得具体的值，而使用.map()方法批量上传时不能使用.getInfo()方法，这里使用for循环上传数据
for index, (key, value) in enumerate(validImageDict.items()):
  out_image_base = value
  image = ee.Image(imageAddCloudMaskList.get(index))
  exportImageToCloudStorage(out_image_base, image)

  # print(index, key, value)
  # 0 2020-09-01 20200901T025549_20200901T030025_T49QHF
  # 1 2020-09-06 20200906T025551_20200906T031003_T49QHF
  # 2 2020-09-11 20200911T025549_20200911T030838_T49QHF
  # 3 2020-09-16 20200916T025551_20200916T030959_T49QHF
  # 4 2020-09-21 20200921T025549_20200921T030950_T49QHF
  # 5 2020-09-26 20200926T025551_20200926T030132_T49QHF



*5.* 获取时间序列数据集

In [14]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=olYh0qLgqsmculXf1u8V721Pedn43r&prompt=consent&access_type=offline&code_challenge=otMkt4qCKnjwtfEUDbrBJyoig81UP8Ft4eumyf8Mw7A&code_challenge_method=S256

Enter authorization code: 4/0AfJohXkPRzAtKltxH54JCJfgvapj7wnwYjm8VgnoKbexZXh6fpjIGqc__FirOQq5M2SltA

You are now logged in as [2200271008@email.szu.edu.cn].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_

In [15]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# # KERNEL_SHAPE = [256, 256]
# # kernel_buffer = [64, 64]
# BANDS = ['B2','B3','B4','B8','cloud_mask']

# # 1.获取TFRecoder数据和json文件
# filesList1 = !gsutil ls 'gs://u-tilise/test'
# filesList1 = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}
# exportFilesList1 = [s for s in filesList1 if '20200901' in s]
# imageFilesList1 = []
# jsonFile1 = None
# for f in exportFilesList1:
#   if f.endswith('.tfrecord.gz'):
#     imageFilesList1.append(f)
#   elif f.endswith('.json'):
#     jsonFile1 = f

# # Make sure the files are in the right order.
# imageFilesList1.sort()

# from pprint import pprint
# pprint(imageFilesList1)
# print(jsonFile1)

# import json
# # Load the contents of the mixer file to a JSON object.
# # TODO:
# jsonText1 = !gsutil cat {jsonFile1}
# # 读取并打印JSON文件内容
# # with open(json_file_path, 'r') as json_file:
# # with open(user_folder+'/fcnn-demo/prediction/'+jsonFile, 'r') as json_file:
# # with open(jsonFile, 'r') as json_file:
# #     jsonText = json_file.read()
# #     print(jsonText)
# # Get a single string w/ newlines from the IPython.utils.text.SList
# mixer1 = json.loads(jsonText1.nlstr)
# # mixer = json.loads(jsonText)
# pprint(mixer1)
# patches = mixer1['totalPatches']

In [16]:
# !pip install rasterio
# import rasterio

from datetime import datetime

In [ ]:
# 获取tif文件的数据，构建时间序列data_torch_tensor
filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}'/shenzhen_tif_202009'
exportFilesList = [s for s in filesList if '202009' in s]
exportFilesList.sort()
imageFilesList = []
imageNumpyList = []
metadataDict = None
for f in exportFilesList:
  if f.endswith('.tif'):
    imageFilesList.append(f)
    # 打开 TIFF 文件
    with rasterio.open(f, 'r') as src:
      # 读取 TIFF 文件中的数据，并添加到 imageNumpyList 中
      tiff_data = src.read()
      imageNumpyList.append(tiff_data)

      # 获取元数据信息，每个tif的metadata都是一样的，只记录一个就行了
      if metadataDict is None:
        metadataDict = src.meta

      # # 打印数据维度和元数据
      # print("Data Type:", type(tiff_data)) # Data Type: <class 'numpy.ndarray'>
      # print("Data Shape:", tiff_data.shape) # Data Shape: (17, 1076, 1102)
      # # 每个tif的metadata都是一样的
      # print("Metadata Type:", type(metadata)) # Metadata Type: <class 'dict'>
      # print("Metadata:", metadata) # Metadata: {'driver': 'GTiff', 'dtype': 'int16', 'nodata': None, 'width': 1102, 'height': 1076, 'count': 17, 'crs': CRS.from_epsg(32649), 'transform': Affine(10.0, 0.0, 801680.0, 0.0, -10.0, 2512970.0)}

# # Make sure the files are in the right order.
imageFilesList.sort()

from pprint import pprint
pprint(imageFilesList)

data_np = np.stack(imageNumpyList, axis=0)
# print(data_np.shape) # (6, 17, 1076, 1102)
# print(type(image_np_data)) # <class 'numpy.ndarray'>
# print(len(imageNumpyList)) # 6

# 将numpy.array转为torch.tensor，并构建成(B,T,C,H,W)的形式
data_torch_tensor = torch.from_numpy(data_np).unsqueeze(0)
# print(type(data_torch_tensor)) # <class 'torch.Tensor'>
# print(data_torch_tensor.shape) # torch.Size([1, 6, 17, 1076, 1102])
# print(data_torch_tensor.dtype) # torch.int16

# 将数据缩小为0到1之间，先除以10000，再将大于1的部分设为1，小于0的部分设为0
data_torch_tensor = torch.clamp(data_torch_tensor / 10000.0, min=0, max=1)
print(data_torch_tensor.min(), data_torch_tensor.max()) # tensor(0.) tensor(1.)

In [ ]:
# 保存data_np为npy文件，以后不用再请求 Cloud Storage
np.save('/content/drive/MyDrive/U-TILISE/shenzhen_tif_202009.npy',data_np)

In [17]:
# 读取npy文件，得到data_np数据
data_np = np.load('/content/drive/MyDrive/U-TILISE/shenzhen_tif_202009.npy')
print(data_np.shape)

data_torch_tensor = torch.from_numpy(data_np).unsqueeze(0)
data_torch_tensor = torch.clamp(data_torch_tensor / 10000.0, min=0, max=1)
print(data_torch_tensor.shape)

(6, 17, 1076, 1102)
torch.Size([1, 6, 17, 1076, 1102])


*6.* 构建 batch 字典的相关参数，以便在模型中使用

需要构建：

1. x：时间序列数据，4 个通道
2. y：与 x 对应的 GT，没有则为 x
3. position_days：时间序列图像中每张图像是当年的第几天
4. days：时间序列图像中每张图像与第一张图像相差的天数
5. cloud_mask：与 x 对应的云掩膜，1 个通道
6. masks：与 cloud_mask 一样
7. masks_valid_obs：表示时间序列中的图像是否可用，0为不可用，1为可用

其他参数暂时不需要构建先。



In [18]:
# x
data_bgr = data_torch_tensor[:, :, 1:4, :, :]
data_nir = data_torch_tensor[:, :, 7, :, :].unsqueeze(2)
x = torch.cat([data_bgr,data_nir],dim=2)
print(data_bgr.shape, data_nir.shape, x.shape) # torch.Size([1, 6, 3, 1076, 1102]) torch.Size([1, 6, 1, 1076, 1102]) torch.Size([1, 6, 4, 1076, 1102])


# position_days
dateList = list(validImageDict.keys())
print(dateList) # ['2020-09-01', '2020-09-06', '2020-09-11', '2020-09-16', '2020-09-21', '2020-09-26']

dates = [float(datetime.strptime(str_date, "%Y-%m-%d").timetuple().tm_yday) for str_date in dateList]
position_days = torch.tensor(np.array(dates)).unsqueeze(0)
# print(position_days) # tensor([[245., 250., 255., 260., 265., 270.]], dtype=torch.float64)


# days
days = [datetime.strptime(str_date, "%Y-%m-%d") for str_date in dateList] # 获取datetime格式的时间，以便进行减法运算
days = [day - days[0] for day in days] # 获得时间间隔
days = [td.days + td.seconds / (24 * 3600) for td in days] # 将时间间隔转为float
# print(days) # [0.0, 5.0, 10.0, 15.0, 20.0, 25.0]

days = torch.tensor(np.array(days)).unsqueeze(0) # 将列表转化为torch.tensor,并构建成(B,T)形式
# print(days) # tensor([[ 0.,  5., 10., 15., 20., 25.]], dtype=torch.float64)


# cloud_mask、maks、maks_valid_obs
cloud_mask = data_torch_tensor[:, :, -1, :, :].unsqueeze(2)
maks = cloud_mask
maks_valid_obs = torch.ones(cloud_mask.shape)
print(cloud_mask.shape, maks_valid_obs.shape) # torch.Size([1, 6, 1, 1076, 1102]) torch.Size([1, 6, 1, 1076, 1102])

torch.Size([1, 6, 3, 1076, 1102]) torch.Size([1, 6, 1, 1076, 1102]) torch.Size([1, 6, 4, 1076, 1102])
['2020-09-01', '2020-09-06', '2020-09-11', '2020-09-16', '2020-09-21', '2020-09-26']
torch.Size([1, 6, 1, 1076, 1102]) torch.Size([1, 6, 1, 1076, 1102])


In [ ]:
my_batch ={}
my_batch['x'] = x
my_batch['y'] = x
my_batch['position_days'] = position_days
my_batch['days'] = days
my_batch['cloud_mask'] = cloud_mask
my_batch['maks'] = maks
my_batch['maks_valid_obs'] = maks_valid_obs
print(my_batch)

In [25]:
# 保存构建好的my_batch
np.save('/content/drive/MyDrive/U-TILISE/my_batch.npy', my_batch)

*7.* 使用模型做预测

In [19]:
# 初始化模型u-tilise
config_file_test = 'configs/config_earthnet2021_test.yaml'
dataloader = get_dataloader_testdata(config_file_train, config_file_test)
utilise = Imputation(config_file_train, method='utilise', checkpoint=checkpoint)

# 获取原始batch
batch = get_sample(dataloader, sample_index=294)

# 将batch中的部分信息用my_batch中的替换
batch['x'] = x.to(torch.float)
batch['y'] = x.to(torch.float)
batch['position_days'] = position_days.to(torch.float)
batch['days'] = days.to(torch.float)
batch['cloud_mask'] = cloud_mask.to(torch.float)
batch['maks'] = maks.to(torch.float)
batch['maks_valid_obs'] = maks_valid_obs.to(torch.float)


NameError: ignored

In [ ]:
# 使用模型预测
_, y_pred = utilise.impute_sample(batch)

# ee.Image.toArray画图测试

In [ ]:
import geemap
# A function to print arrays for a selected pixel in the following examples.
def samp_arr_img(arr_img):
  point = ee.Geometry.Point([-121, 42])
  return arr_img.sample(point, 500).first().get('array')


# A 3-band image of constants.
# img = ee.Image([1, 2, 3])
img = collection.first().select(['B4','B3','B2'])
display('3-band image', img)

# Convert the 3-band image to an array image. The resulting array image has a
# single band named "array". The "array" band stores the per-pixel band values
# from the input ee.Image as a 1D array.
array_img_1_d = img.toArray()
display('1D array image', array_img_1_d)

# Sample a single pixel to see its 1D array using the `samp_arr_img` function
# defined above. Similar arrays are present for all pixels in a given array
# image looking at just one helps conceptualize the structure.
display('1D array image (pixel)', samp_arr_img(array_img_1_d))
# [0, 1, 2]

# Array images can be displayed to the Code Editor map and queried with the
# inspector tool. Per-pixel, the first element in the array is shown.
# Single-band grayscale is used to represent the data. Style parameters `min`
# and `max` are valid. Styling the layer with the Code Editor's layer
# visualization tool is invalid.
m = geemap.Map()
m.add_layer(array_img_1_d, {'min': 0, 'max': 3000}, 'Image array')
# display(m)

# Create a 2D array image by concatenating the values in a 1D array image
# along the 1-axis using `toArray(1)`. For a 3D array, apply `toArray(2)` to
# the result.
array_img_2_d = array_img_1_d.toArray(1)
display('2D array image (pixel)', samp_arr_img(array_img_2_d))
# [[0],
#  [1],
#  [2]]
m.add_layer(array_img_2_d, {'min': 0, 'max': 3000}, 'Image array2')
display(m)

'3-band image'

'1D array image'

'1D array image (pixel)'

'2D array image (pixel)'

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…